In [1]:
# This piece of code enables display of multiple output from one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Data Description

The data has been split into two groups:

training set (train.csv)
test set (test.csv)
The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

Data Dictionary
Variable	Definition	Key
survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes
pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.


Reference: [Kaggle](https://www.kaggle.com/c/titanic/data?select=train.csv)

## Classification Model Fitting, Evaluation and Selection

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import metrics

In [5]:
df = pd.read_csv("titanic_train.csv") 
df_test = pd.read_csv("titanic_test.csv")
df.shape
df_test.shape

(891, 12)

(418, 11)

In [6]:
df.head().T

,0,1,2,3,4
PassengerId,1,2,3,4,5
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,22.0,38.0,26.0,35.0,35.0
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450
Fare,7.25,71.2833,7.925,53.1,8.05


### Check Data Quality

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
# Drop extraneous columns
df = df.drop(['Ticket','Cabin'], axis=1)

In [9]:
# Impute median Age for NA Age values
imputed_age = df["Age"].median()
new_age_var = np.where(df["Age"].isnull(), imputed_age, df["Age"])

df["Age"] = new_age_var

In [10]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
dtype: int64

In [11]:
# Remove NaN values
df = df.dropna()
df.shape

(889, 10)

### Perform EDA
Leaving it to the students to perform the EDA on this dataset before fitting the model

### Get the data ready for fitting a model

In [12]:
df_train = df.copy()
y_train = df_train.Survived.values
del df_train['Survived']

In [13]:
df_train.head().T


,0,1,2,3,4
PassengerId,1,2,3,4,5
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,22.0,38.0,26.0,35.0,35.0
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Fare,7.25,71.2833,7.925,53.1,8.05
Embarked,S,C,S,S,S


In [14]:
categorical = ['Pclass', 'Sex', 'Embarked']
numerical = ['Age', 'Fare', 'SibSp', 'Parch']

In [15]:
# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()


encoded_sex = label_encoder.fit_transform(df_train["Sex"])
encoded_class = label_encoder.fit_transform(df_train["Pclass"])


train_features = pd.DataFrame([encoded_class,
                              encoded_sex,
                              df_train["Age"]]).T

model_ = LogisticRegression(solver = 'lbfgs')

model_.fit(X = train_features, y = y_train)

print(model_.intercept_)

print(model_.coef_)


# Model accuracy
model_.score(X = train_features , y = y_train)

LogisticRegression()

[3.44087935]
[[-1.13805975 -2.51240389 -0.03297024]]


0.7874015748031497

In [16]:
preds = model_.predict(X= train_features)
metrics.confusion_matrix(y_true=y_train, y_pred=preds)
print(metrics.classification_report(y_true=y_train, y_pred=preds) )

array([[454,  95],
       [ 94, 246]])

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       549
           1       0.72      0.72      0.72       340

    accuracy                           0.79       889
   macro avg       0.77      0.78      0.78       889
weighted avg       0.79      0.79      0.79       889



In [17]:
df_test = df_test.drop(['Ticket','Cabin'], axis=1)
new_age_var = np.where(df_test["Age"].isnull(), imputed_age, df_test["Age"])
df_test["Age"] = new_age_var

In [18]:
X_test = df_test
y_test = df_test.Survived.values
del X_test['Survived']

AttributeError: 'DataFrame' object has no attribute 'Survived'

In [19]:
# Split into test and train
# Split into train and validate
df_train, df_val = train_test_split(df, test_size=0.33, random_state=11)

y_train = df_train.Survived.values
y_val = df_val.Survived.values

del df_train['Survived']
del df_val['Survived']

In [20]:
# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()


encoded_sex = label_encoder.fit_transform(df_train["Sex"])
encoded_class = label_encoder.fit_transform(df_train["Pclass"])


train_features = pd.DataFrame([encoded_class,
                              encoded_sex,
                              df_train["Age"]]).T

model_ = LogisticRegression(solver = 'lbfgs')

model_.fit(X = train_features, y = y_train)

print(model_.intercept_)

print(model_.coef_)


# Model accuracy
model_.score(X = train_features , y = y_train)

LogisticRegression()

[3.15738506]
[[-1.1033871  -2.25572176 -0.02900607]]


0.7596638655462185

In [21]:
encoded_sex = label_encoder.fit_transform(df_val["Sex"])
encoded_class = label_encoder.fit_transform(df_val["Pclass"])
X_val = pd.DataFrame([encoded_class, encoded_sex, df_val["Age"]]).T

In [22]:
preds = model_.predict(X= X_val)
metrics.confusion_matrix(y_true=y_val, y_pred=preds)
print(metrics.classification_report(y_true=y_val, y_pred=preds) )

array([[152,  28],
       [ 24,  90]])

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       180
           1       0.76      0.79      0.78       114

    accuracy                           0.82       294
   macro avg       0.81      0.82      0.81       294
weighted avg       0.82      0.82      0.82       294



In [23]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)

DictVectorizer(sparse=False)

In [24]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [25]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)[:, 1]

## Accuracy

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]
churn = y_pred >= 0.5
(churn == y_val).mean()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_val, y_pred >= 0.5)

In [ ]:
thresholds = np.linspace(0, 1, 11)
thresholds

In [ ]:
thresholds = np.linspace(0, 1, 21)

accuracies = []

for t in thresholds:
    acc = accuracy_score(y_val, y_pred >= t)
    accuracies.append(acc)
    print('%0.2f %0.3f' % (t, acc))

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(thresholds, accuracies, color='black')

plt.title('Threshold vs Accuracy')
plt.xlabel('Threshold')
plt.ylabel('Accuracy')

plt.xticks(np.linspace(0, 1, 11))

# plt.savefig('04_threshold_accuracy.svg')

plt.show()


## Confusion Table

In [ ]:
true_positive = ((y_pred >= 0.6) & (y_val == 1)).sum()
false_positive = ((y_pred >= 0.6) & (y_val == 0)).sum()
false_negative = ((y_pred < 0.6) & (y_val == 1)).sum()
true_negative = ((y_pred < 0.6) & (y_val == 0)).sum()

In [ ]:
confusion_table = np.array(
     # predict neg    pos
    [[true_negative, false_positive], # actual neg
     [false_negative, true_positive]]) # actual pos

confusion_table

In [ ]:
confusion_table / confusion_table.sum()

## Precision Recall

In [ ]:
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
precision, recall

## ROC and AUC

In [ ]:
scores = []

thresholds = np.linspace(0, 1, 101)

for t in thresholds: #B
    tp = ((y_pred >= t) & (y_val == 1)).sum()
    fp = ((y_pred >= t) & (y_val == 0)).sum()
    fn = ((y_pred < t) & (y_val == 1)).sum()
    tn = ((y_pred < t) & (y_val == 0)).sum()
    scores.append((t, tp, fp, fn, tn))

df_scores = pd.DataFrame(scores)
df_scores.columns = ['threshold', 'tp', 'fp', 'fn', 'tn']

In [ ]:
df_scores[::10]

In [ ]:
df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(df_scores.threshold, df_scores.tpr, color='black', linestyle='solid', label='TPR')
plt.plot(df_scores.threshold, df_scores.fpr, color='black', linestyle='dashed', label='FPR')
plt.legend()

plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, 1, 11))

plt.xlabel('Thresholds')
plt.title('TPR and FPR')

# plt.savefig('04_fpr_tpr_plot.svg')

plt.show()

## ROC

In [ ]:
def tpr_fpr_dataframe(y_val, y_pred):
    scores = []

    thresholds = np.linspace(0, 1, 101)

    for t in thresholds:
        tp = ((y_pred >= t) & (y_val == 1)).sum()
        fp = ((y_pred >= t) & (y_val == 0)).sum()
        fn = ((y_pred < t) & (y_val == 1)).sum()
        tn = ((y_pred < t) & (y_val == 0)).sum()

        scores.append((t, tp, fp, fn, tn))

    df_scores = pd.DataFrame(scores)
    df_scores.columns = ['threshold', 'tp', 'fp', 'fn', 'tn']

    df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
    df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)

    return df_scores

In [ ]:
np.random.seed(1)
y_rand = np.random.uniform(0, 1, size=len(y_val))
df_rand = tpr_fpr_dataframe(y_val, y_rand)
df_rand[::10]

In [ ]:
num_neg = (y_val == 0).sum()
num_pos = (y_val == 1).sum()

y_ideal = np.repeat([0, 1], [num_neg, num_pos])
y_pred_ideal = np.linspace(0, 1, num_neg + num_pos)

df_ideal = tpr_fpr_dataframe(y_ideal, y_pred_ideal)
df_ideal[::10]

In [ ]:



plt.figure(figsize=(5, 5))

plt.plot(df_scores.fpr, df_scores.tpr, color='black', label='Model')
plt.plot(df_rand.fpr, df_rand.tpr, color='black', lw=1,
         linestyle='dashed', alpha=0.5, label='Random')
plt.plot(df_ideal.fpr, df_ideal.tpr, color='black', lw=0.5,
         linestyle='solid', alpha=0.5, label='Ideal')

plt.legend()

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('ROC curve')

# plt.savefig('04_roc_curve_with_baselines.svg')

plt.show()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_pred)

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(fpr, tpr, color='black')
plt.plot([0, 1], [0, 1], color='black', lw=0.7, linestyle='dashed', alpha=0.5)

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('ROC curve')

plt.show()

In [ ]:
auc(df_scores.fpr, df_scores.tpr)

## K-fold cross-validation


In [ ]:
def train(df, y):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear')
    model.fit(X, y)

    return dv, model


def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
df.head().T

In [ ]:
aucs = []

for train_idx, val_idx in kfold.split(df):
    df_train = df.iloc[train_idx]
    y_train = df_train.Survived.values

    df_val = df.iloc[val_idx]
    y_val = df_val.Survived.values

    dv, model = train(df_train, y_train)
    y_pred = predict(df_val, dv, model)

    rocauc = roc_auc_score(y_val, y_pred)
    aucs.append(rocauc)

In [ ]:
np.array(aucs).round(3)

# Tuning the L2 Hyperparameter

In [ ]:
def train(df, y, C=1.0):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear', C=C)
    model.fit(X, y)

    return dv, model

In [ ]:
nfolds = 5
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=1)

for C in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    aucs = []

    for train_idx, val_idx in kfold.split(df):
        df_train = df.iloc[train_idx]
        df_val = df.iloc[val_idx]

        y_train = df_train.Survived.values
        y_val = df_val.Survived.values

        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)
        
        auc = roc_auc_score(y_val, y_pred)
        aucs.append(auc)

    print('C=%s, auc = %0.3f ± %0.3f' % (C, np.mean(aucs), np.std(aucs)))

In [ ]:
df_test.isnull().sum()
df_test = df_test.dropna()

In [ ]:
y_train = df.Survived.values


dv, model = train(df, y_train, C=0.1)

y_pred = predict(df_test, dv, model)

### Reference
[Dictionary in Pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)

[Kaggle Titanic](https://nbviewer.org/github/agconti/kaggle-titanic/blob/master/Titanic.ipynb)

[Scikit-learn User Guide](https://scikit-learn.org/stable/user_guide.html)

[Scikit-learn Preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html)

[Scikit-learn Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection)